In [524]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor

In [525]:
class RotationTranslationMatrix:
    def __init__(self, m) -> None:
        assert m.shape[0] == m.shape[1]
        self.m = m

In [526]:
class PointV:
    def __init__(self, *xyz, v=None):
        if xyz:
            self.v = np.array(xyz) 
        else:
            self.v = v

    def v_affine(self):
        return self.v[:-1] / self.v[-1]

    def v_homogenious(self):
        return np.append(self.v, 1) 
    
    def __hash__(self) -> int:
        return hash(self.v.tobytes())

    def __eq__(self, __o):
        return np.all(self.v == __o.v)

    def __add__(self, __o):
        if isinstance(__o, PointV):
            return PointV(v=self.v + __o.v)
        return PointV(v=self.v + __o)

    def __sub__(self, __o):
        return PointV(v=self.v - __o.v)

    def __mul__(self, value):
        return PointV(v=self.v * value)
    
    def __truediv__(self, value):
        return PointV(v=self.v / value)
    
    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()

    def __matmul__(self, m):
        return PointV(self.v_homogenious() @ m).v_affine()

    def __repr__(self) -> str:
        return f"PointV {self.v}"

In [527]:
class Edge:
    def __init__(
        self,
        a,
        b,
    ):
        self.a = a
        self.b = b
        self.v = b - a
        self.normal = None

    def __eq__(self, __o):
        return (
            self.a == __o.a and self.b == __o.b or \
                self.a == __o.b and self.b == __o.a
        )
    
    def __hash__(self) -> int:
        a_bytes = self.a.v.tobytes()
        b_bytes = self.b.v.tobytes()
        return hash(tuple(sorted((a_bytes, b_bytes))))

    def normal_to(self, viewpoint):
        v_a = viewpoint - self.a
        v_b = self.b - self.a
        proj = v_b * np.dot(v_a.v, v_b.v) / v_b.norm() ** 2
        n = (proj - v_a).unit() 
        return n

    def __repr__(self) -> str:
        return f"Edge <{self.a} {self.b}>"

In [528]:
# TODO
# Polygon -> SimplePolygon -> Triangle

import functools
import operator
from ordered_set import OrderedSet

class Polygon:
    ...

class SimplePolygon(Polygon):
    ...

class Triangle(SimplePolygon):
    ...

class Polygon:
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.simple_type = None
        self.triangluation = []
        self.simple_polygon_list = []
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, f"{self.__class__.__name__}: vertices and edges are empty"

        if self.vertices is None:
            self._vertices()

        if self.edges is None:
            self._edges()

        if self.L is None:
            self.L = len(self.vertices)

        if self.convex_type is None:
            self._convex_type()

        if self.simple_type is None:
            self._simple_type()

        if self.simple_polygon_list == []:
            self._simple_polygon_list()

        if self.rotation is None:
            self._rotation()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _edges(self):
        self.edges = OrderedSet(
            Edge(p1, p2)
            for p1, p2 in zip(self.vertices, self.vertices[1:] + [self.vertices[0]])
        )

    def _vertices(self):
        self.vertices = tuple(edge.a for edge in self.edges)

    def _convex_type(self):
        flag = None
        for i in range(self.L):
            for j in range(i + 1, i + self.L - 1):
                cl_type = classifyEdgePointV(self.edges[j % self.L], self.vertices[i])
                if flag is None:
                    flag = cl_type
                elif flag != cl_type:
                    self.convex_type = "CONCAVE"
                    return
        self.convex_type = "CONVEX"

    def _simple_type(self):
        flag = "SIMPLE"
        intersection_pairs = {}
        for i in range(self.L - 1):
            for j in range(i + 2, self.L):
                if i == (j + 1) % self.L:
                    continue
                vi, vj = self.edges[i], self.edges[j]
                cross_type, ti = crossEdges(vi, vj)
                if cross_type == "SKEW_CROSS":
                    intersection_PointV = vi.a + (vi.b - vi.a) * ti
                    tj = (intersection_PointV - vj.a).norm() / vj.v.norm()
                    intersection_pairs[(i, j)] = (intersection_PointV, ti, tj)
                    flag = "SELF_INTERSECTED"
        self.simple_type = flag
        self._intersection_pairs = intersection_pairs

    def _rotation(self):
        self.rotation = [x.rotation for x in self.simple_polygon_list]

    def _simple_polygon_list(self):
        # Step 1: Collect intersection PointVs per edge
        splits = {}  # edge_idx -> [(t_param, PointV), ...]
        for (i, j), (PointV, ti, tj) in self._intersection_pairs.items():
            for idx, t in ((i, ti), (j, tj)):
                splits.setdefault(idx, []).append((t, PointV))

        # Sort splits by parameter t for each edge
        for split_list in splits.values():
            split_list.sort(key=lambda x: x[0])

        # Build new edges from sorted splits
        split_edges = {}
        for idx, split_list in splits.items():
            edge = self.edges[idx]
            segments = [edge.a] + [pt for _, pt in split_list] + [edge.b]
            split_edges[idx] = [
                Edge(segments[i], segments[i + 1]) for i in range(len(segments) - 1)
            ]

        stack_edges = [[]]
        for i in range(self.L):
            if i not in split_edges:
                stack_edges[-1].append(self.edges[i])
                if (
                    len(stack_edges[-1]) > 1 and \
                        stack_edges[-1][0].a == stack_edges[-1][-1].b
                ):
                    self.simple_polygon_list.append(
                        SimplePolygon(
                            edges=stack_edges.pop(), 
                            color=self.color
                            )
                        )
            else:
                for edge in split_edges[i]:
                    stack_edges[-1].append(edge)
                    if (
                        len(stack_edges[-1]) > 1 and \
                            stack_edges[-1][0].a == stack_edges[-1][-1].b
                    ):
                        self.simple_polygon_list.append(
                            SimplePolygon(
                                edges=stack_edges.pop(), 
                                color=self.color
                                )
                            )
                    else:
                        stack_edges.append([])
                if stack_edges:
                    del stack_edges[-1]

    def EO(self, p):
        intersection_sum = 0
        for edge in self.edges:
            match classifyEdgePointV(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    intersection_sum += 1
                case "CROSS_RIGHT":
                    intersection_sum += 1
        if intersection_sum % 2 == 1:
            return "INSIDE"
        return "OUTSIDE"

    def NZW(self, p):
        winding_number = 0
        for edge in self.edges:
            match classifyEdgePointV(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    winding_number += 1
                case "CROSS_RIGHT":
                    winding_number -= 1
        if winding_number == 0:
            return "OUTSIDE"
        return "INSIDE"

In [529]:
class SimplePolygon(Polygon):
    
    def __init__(self, **kwargs) -> None:
        
        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.signed_area = None
        self.triangulation = []
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, f"{self.__class__.__name__}: vertices and edges are empty"

        if self.vertices is None:
            self._vertices()

        if self.edges is None:
            self._edges()

        if self.L is None:
            self.L = len(self.vertices)

        if self.convex_type is None:
            self._convex_type()
        
        if self.triangulation == []:
            self._triangulation()

        if self.rotation is None:
            self._rotation()

    def _signed_area(self):
        self.signed_area = 0.5 * sum(np.cross(edge.a.v, edge.b.v) for edge in self.edges)

    def _rotation(self):
        self.rotation = self.triangulation[0].rotation

    def join(self, other: Polygon):
        if self.rotation != other.rotation:
            for edge in other.edges:
                edge = Edge(edge.b, edge.a)
        self.edges = self.edges.symmetric_difference(other.edges)
        self._vertices() 
        
    def _triangulation(self):
        sorted_PointVs = sorted(self.vertices, key=lambda x: x.v[0])
        triangle = Triangle(vertices=sorted_PointVs[:3])
        current = triangle
        self.triangulation.append(current)
        for viewPointV in sorted_PointVs[3:]:
            for edge in current.edges:
                if edge_visible(edge, edge.a - viewPointV):
                    triangle = Triangle(vertices=[edge.b, edge.a, viewPointV])
                    self.triangulation.append(triangle)
                    current.join(triangle)


In [530]:
class Triangle(SimplePolygon):
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.signed_area = None
        self.centroid = None
        self.edge_normals = None
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Triangle: vertices and edges are empty"

        if self.edges is None:
            self._edges()

        if self.vertices is None:
            self._vertices()

        if self.centroid is None:
            self._centroid()
        
        if self.signed_area is None:
            self._signed_area()

        if self.rotation is None:
            self._rotation()

        if self.edge_normals is None:
            self._edge_normals()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _rotation(self):
        self.rotation = "CCW" if self.signed_area > 0 else "CW"

    def _centroid(self):
        self.centroid = functools.reduce(operator.add, self.vertices) / 3
    
    def _edge_normals(self):
        for edge in self.edges:
            edge.normal = edge.normal_to(self.centroid)


In [531]:
def edge_visible(edge, view):
    dot_product = np.dot(edge.normal, view.v)
    return dot_product < 0


In [532]:
def draw_polygon(polygon, draw):
    for edge in polygon.edges:
        draw.point(pixelEdge(edge), fill=polygon.color)

In [533]:
def paint_polygon(polygon, draw):
    for simple_poly in polygon.simple_polygon_list:
        for triangle in simple_poly.triangulation:
            draw.point(paint_triangle(triangle), fill="red")

In [534]:
def pixelLine(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    ix = 1 if dx > 0 else -1
    iy = 1 if dy > 0 else -1
    dx *= ix
    dy *= iy
    x = x1
    y = y1
    pixels = []
    if dx < dy:
        E = 2 * dx - dy
        for _ in range(dy + 1):
            pixels.append((x, y))
            if E >= 0:
                x += ix
                E -= 2 * dy
            y += iy
            E += 2 * dx
    else:
        E = 2 * dy - dx
        for _ in range(dx + 1):
            pixels.append((x, y))
            if E >= 0:
                y += iy
                E -= 2 * dx
            x += ix
            E += 2 * dy
    return pixels


def pixelEdge(e):
    x1, y1, x2, y2 = (
        round(e.a.v[0]),
        round(e.a.v[1]),
        round(e.b.v[0]),
        round(e.b.v[1]),
    )
    return pixelLine(x1, y1, x2, y2)

In [535]:
def classify(x1, y1, x2, y2, x, y):
    ax = x2 - x1
    ay = y2 - y1
    bx = x - x1
    by = y - y1
    s = ax * by - ay * bx
    if s > 0:
        return "LEFT"
    if s < 0:
        return "RIGHT"
    if ax * bx < 0 or ay * by < 0:
        return "BEHIND"
    if ax**2 + ay**2 < bx**2 + by**2:
        return "INFRONT"
    if x == x1 and y == y1:
        return "ORIGIN"
    if x == x2 and y == y2:
        return "DESTINATION"
    return "BETWEEN"


def classifyEdgePointV(e, p):
    return classify(*e.a.v, *e.b.v, *p.v)

In [536]:
def intersect(ax, ay, bx, by, cx, cy, dx, dy):
    nx = dy - cy
    ny = cx - dx
    denom = nx * (bx - ax) + ny * (by - ay)
    if denom == 0:
        type = classify(cx, cy, dx, dy, ax, ay)
        if type == "LEFT" or type == "RIGHT":
            return "PARALLEL", None
        else:
            return "SAME", None
    num = nx * (ax - cx) + ny * (ay - cy)
    t = -num / denom
    return "SKEW", t


def intersectEdges(e1, e2):
    return intersect(*e1.a.v, *e1.b.v, *e2.a.v, *e2.b.v)

In [537]:
def cross(ax, ay, bx, by, cx, cy, dx, dy):
    type, tab = intersect(ax, ay, bx, by, cx, cy, dx, dy)
    if type == "SAME" or type == "PARALLEL" or tab is None:
        return type, None
    if tab < 0 or tab > 1:
        return "SKEW_NO_CROSS", None
    _, tcd = intersect(cx, cy, dx, dy, ax, ay, bx, by)
    if tcd is not None and (tcd < 0 or tcd > 1):
        return "SKEW_NO_CROSS", None
    return "SKEW_CROSS", tab


def crossEdges(e1, e2):
    return cross(*e1.a.v, *e1.b.v, *e2.a.v, *e2.b.v)

In [538]:
def paint_triangle(triangle):
    edges = [pixelEdge(e) for e in sorted(triangle.edges, key=lambda x: x.norm())]
    ABC = edges[0] + edges[1]
    AC = edges[2][::-1]

    A = AC[0]
    C = AC[-1]
    dxAC = abs(A[0] - C[0])
    dyAC = abs(A[1] - C[1])
    k = 0 if dxAC > dyAC else 1  # 0 = iterate by x, 1 = iterate by y

    pixels = []
    j = 0

    for i in range(len(ABC)):
        # advance j until AC[j] matches ABC[i] on major axis
        while j + 1 < len(AC) and AC[j+1][k] <= ABC[i][k]:
            j += 1

        if j >= len(AC):
            break

        line = pixelLine(*ABC[i], *AC[j])
        pixels.extend(line)

        if ABC[i] == C:
            break

    return pixels


In [539]:
img = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(img)

In [540]:
p0 = Polygon(
    vertices=[
        PointV(150, 80),
        PointV(10, 60),
        PointV(50, 100),
        PointV(100, 20),
        PointV(120, 160),
        PointV(190, 100),
        PointV(180, 190),
    ],
    color="blue",
)

In [541]:
e = Edge(PointV(10,10), PointV(20,20))
e.normal_to(PointV(20, 10))

array([-0.70710678,  0.70710678])

In [542]:
p0.convex_type

'CONCAVE'

In [543]:
p0.simple_type

'SELF_INTERSECTED'

In [544]:
p0.simple_polygon_list

In [545]:
p0.rotation

['CW', 'CCW', 'CW', 'CW']

In [551]:
for sp in p0.simple_polygon_list:
    for tri in sp.triangulation:
        print(tri.edges)

OrderedSet([Edge <PointV [10 60] PointV [ 50 100]>, Edge <PointV [ 50 100] PointV [69.67213115 68.52459016]>, Edge <PointV [69.67213115 68.52459016] PointV [10 60]>])
OrderedSet([Edge <PointV [69.67213115 68.52459016] PointV [100  20]>, Edge <PointV [100  20] PointV [107.70833333  73.95833333]>, Edge <PointV [107.70833333  73.95833333] PointV [69.67213115 68.52459016]>])
OrderedSet([Edge <PointV [162. 124.] PointV [180 190]>, Edge <PointV [180 190] PointV [190 100]>, Edge <PointV [190 100] PointV [162. 124.]>])
OrderedSet([Edge <PointV [107.70833333  73.95833333] PointV [120 160]>, Edge <PointV [150  80] PointV [107.70833333  73.95833333]>, Edge <PointV [120 160] PointV [162. 124.]>, Edge <PointV [162. 124.] PointV [150  80]>])
OrderedSet([Edge <PointV [150  80] PointV [120 160]>, Edge <PointV [120 160] PointV [162. 124.]>, Edge <PointV [162. 124.] PointV [150  80]>])


In [552]:
p0.vertices

[PointV [150  80],
 PointV [10 60],
 PointV [ 50 100],
 PointV [100  20],
 PointV [120 160],
 PointV [190 100],
 PointV [180 190]]

In [554]:
draw_polygon(p0, draw)
img.show()

In [ ]:
# paint_polygon(p0, draw)

AttributeError: 'Edge' object has no attribute 'norm'

In [ ]:
p1 = Polygon(
    vertices=[
        PointV(10, 10),
        PointV(40, 100),
        PointV(100, 150),
        PointV(50, 180),
        PointV(70, 10),
    ],
    color="green",
)

In [ ]:
p1.convex_type

'CONCAVE'

In [ ]:
p1.simple_type

'SELF_INTERSECTED'

In [ ]:
p1.simple_polygon_list

In [ ]:
# draw_polygon(p1, draw)

In [ ]:
p2 = Polygon(
    vertices=[
        PointV(20, 10),
        PointV(50, 10),
        PointV(80, 20),
        PointV(90, 50),
        PointV(70, 80),
        PointV(40, 90),
        PointV(10, 60),
    ],
    color="yellow",
)

In [ ]:
p2.convex_type

'CONVEX'

In [ ]:
p2.simple_type

'SIMPLE'

In [ ]:
p2.simple_polygon_list

In [ ]:
# draw_polygon(p2, draw)
# img.show()

In [ ]:
triangle = Polygon(
    vertices=[
        PointV(34, 80),
        PointV(57, 1),
        PointV(170, 80),
    ],
    color="yellow",  # A B  # B C  # C A
)

In [ ]:
# draw.PointV(paint_triangle(), fill="red")


img.show()